# Use DLite strategies from OTEAPI-OPTIMADE

This example shows how to use the DLite strategies from the OTEAPI-OPTIMADE plugin.

[DLite](https://github.com/SINTEF/dlite) is a Python library for working with data models and semantics.
It is considered to be the default semantic data model backend for use with OTEAPI.

To see more fundamental examples of how to use OTEAPI-OPTIMADE, see the example [_Use OTEAPI-OPTIMADE with OTElib_](/oteapi-optimade/examples/otelib/).
OTElib will also be used as a client in the current example.
Furthermore, only the HTTP requests-based backend will be used in this example.

## Setup

Please see the [setup instructions in _Use OTEAPI-OPTIMADE with OTElib_](/oteapi-optimade/examples/otelib/#setup) for how to ensure you have a proper environment to run the example in.

## Example

In this example, we will use the DLite strategies to query the [Materials Project](https://www.materialsproject.org/) OPTIMADE API.
We are interested in finding all structures that include the elements `Si` and `O`, and that have a maximum of 4 elements in total.

### Create a client

Let's start by initializing a client:

In [37]:
from otelib import OTEClient

client = OTEClient("python")

### Data Resource strategy

The general pipeline is the same as it was for [_Use OTEAPI-OPTIMADE with OTElib_](otelib##search-through-materials-project-for-all-structures-with-the-formula-al2o3):

<!-- The mkdocs-jupyter plugin creates a folder 'dlite' wherein it puts the generated HTML
Therefore, we need to go up one more folder level to reference the image.
One could use an "absolute" link, however, in production, mike will append the version to the path,
which this link could not dynamically include. -->
![Pipeline example 1](../../img/chem_formula_pipeline.png)

However, in order to use DLite we need to _either_ reference a specific `accessService` **or** set a flag in the configuration of the data resource strategy.

Note, it is _only_ for the data resource strategy we need to specify the usage of DLite, as the filter strategy is generic and merely a helper for more explicitly setting the query parameters to be used for the underlying OPTIMADE query.

In [38]:
data_resource_strategy = client.create_dataresource(
    accessService="OPTIMADE+DLite",
    accessUrl="https://optimade.materialsproject.org",
)

# This is equivalent to:
# data_resource_strategy = client.create_dataresource(
#     accessService="OPTIMADE",
#     accessUrl="https://optimade.materialsproject.org",
#     configuration={"use_dlite": True},
# )

### Filter strategy

The OPTIMADE filter query to fulfill [the interests outlined above](#example) should look like this:

```
elements HAS ALL "Si","O" AND nelements<=4
```

In [39]:
filter_strategy = client.create_filter(
    filterType="OPTIMADE",
    query='elements HAS ALL "Si","O" AND nelements<=4',
)

#### Setup, execute and inspect the pipeline

In [40]:
import json

pipeline = filter_strategy >> data_resource_strategy
session = pipeline.get()
parsed_session = json.loads(session)
parsed_session.keys()

Setting filter from query.
Setting filter from query.
Setting filter from query.
resource_config: OPTIMADEResourceConfig(user=None, password=None, token=None, client_id=None, client_secret=None, configuration=OPTIMADEConfig(version='v1', endpoint='structures', query_parameters=OPTIMADEQueryParameters(filter='elements HAS ALL "Si","O" AND nelements<=4', response_format='json', email_address='', response_fields='', sort='', page_limit=20, page_offset=0, page_number=None, page_cursor=0, page_above=None, page_below=None, include='references', api_hint=''), datacache_config=DataCacheConfig(cacheDir=PosixPath('oteapi'), accessKey=None, hashType='md5', expireTime=86400, tag='optimade'), return_object=False, use_dlite=False), description='Resource Strategy Data Configuration.\n\n    Important:\n        Either of the pairs of attributes `downloadUrl`/`mediaType` or\n        `accessUrl`/`accessService` MUST be specified.\n\n    ', downloadUrl=None, mediaType=None, accessUrl=OPTIMADEUrl('https://

dict_keys(['optimade_config', 'optimade_resources', 'optimade_resource_model', 'collection_id'])

In [41]:
from importlib import import_module

import_path, class_name = parsed_session["optimade_resource_model"].split(":", maxsplit=1)
ResourceClass = getattr(import_module(import_path), class_name)

parsed_structures = [ResourceClass(structure) for structure in parsed_session["optimade_resources"]]
print(f"The query resulted in {len(parsed_structures)} structures found (on page 1) of the returned data.")
print(f"Their Materials Project IDs are: {[structure.id for structure in parsed_structures]}")

The query resulted in 20 structures found (on page 1) of the returned data.
Their Materials Project IDs are: ['mp-1033911', 'mp-757887', 'mp-1219366', 'mp-733539', 'mp-542090', 'mp-758465', 'mp-560675', 'mp-774171', 'mp-1304778', 'mp-1016821', 'mp-1363556', 'mp-757013', 'mp-683953', 'mp-1020609', 'mp-17612', 'mp-558129', 'mp-1210628', 'mp-1197149', 'mp-21791', 'mp-752892']


In [42]:
from oteapi_dlite.utils import get_collection

collection = get_collection(session=parsed_session)
print(collection)

{
  "bfd78bf8-31fe-4487-a4c3-8cc5351ca87d":  {
    "meta": "http://onto-ns.com/meta/0.1/Collection",
    "dimensions": {
      "nrelations": 60
    },
    "properties": {
      "relations": [["mp-1033911", "_is-a", "Instance"], ["mp-1033911", "_has-uuid", "f0676948-f620-4057-9291-b1851f519d66"], ["mp-1033911", "_has-meta", "http://onto-ns.com/meta/1.0/OPTIMADEStructure"], ["mp-757887", "_is-a", "Instance"], ["mp-757887", "_has-uuid", "76add43f-bb1e-4bdb-984e-dc6c16b9205d"], ["mp-757887", "_has-meta", "http://onto-ns.com/meta/1.0/OPTIMADEStructure"], ["mp-1219366", "_is-a", "Instance"], ["mp-1219366", "_has-uuid", "d0e7c4f7-1dd7-4206-aa53-2a67b755259d"], ["mp-1219366", "_has-meta", "http://onto-ns.com/meta/1.0/OPTIMADEStructure"], ["mp-733539", "_is-a", "Instance"], ["mp-733539", "_has-uuid", "e460fb2d-1639-4b65-9e61-f4c2869ef412"], ["mp-733539", "_has-meta", "http://onto-ns.com/meta/1.0/OPTIMADEStructure"], ["mp-542090", "_is-a", "Instance"], ["mp-542090", "_has-uuid", "4625be3d-cad7-4

In [43]:
for inst in collection.get_instances():
    print(inst)

{
  "f0676948-f620-4057-9291-b1851f519d66":  {
    "meta": "http://onto-ns.com/meta/1.0/OPTIMADEStructure",
    "dimensions": {
    },
    "properties": {
      "type": "structures",
      "attributes": "46900658-13b0-4072-8fc1-7e20de603765",
      "id": "mp-1033911"
    }
  }
}
{
  "76add43f-bb1e-4bdb-984e-dc6c16b9205d":  {
    "meta": "http://onto-ns.com/meta/1.0/OPTIMADEStructure",
    "dimensions": {
    },
    "properties": {
      "type": "structures",
      "attributes": "62d27df4-ac40-4784-a5fd-ce48cbf12a3a",
      "id": "mp-757887"
    }
  }
}
{
  "d0e7c4f7-1dd7-4206-aa53-2a67b755259d":  {
    "meta": "http://onto-ns.com/meta/1.0/OPTIMADEStructure",
    "dimensions": {
    },
    "properties": {
      "type": "structures",
      "attributes": "c890c277-74b5-42a9-8bed-d0687b34bbd6",
      "id": "mp-1219366"
    }
  }
}
{
  "e460fb2d-1639-4b65-9e61-f4c2869ef412":  {
    "meta": "http://onto-ns.com/meta/1.0/OPTIMADEStructure",
    "dimensions": {
    },
    "properties": {
      

In [44]:
import dlite

structure_instances: list[dlite.Instance] = list(collection.get_instances())

structure_attributes_instance: dlite.Instance = dlite.get_instance(structure_instances[0].attributes)

print(structure_attributes_instance)

{
  "46900658-13b0-4072-8fc1-7e20de603765":  {
    "meta": "http://onto-ns.com/meta/1.0/OPTIMADEStructureAttributes",
    "dimensions": {
      "nelements": 4,
      "dimensionality": 3,
      "nsites": 32,
      "nspecies": 4,
      "nstructure_features": 0
    },
    "properties": {
      "elements": ["K", "Mg", "O", "Si"],
      "nelements": 4,
      "elements_ratios": [0.03125, 0.4375, 0.03125, 0.5],
      "chemical_formula_descriptive": "KMg14O16Si",
      "chemical_formula_reduced": "KMg14O16Si",
      "chemical_formula_hill": "KMg14O16Si",
      "chemical_formula_anonymous": "A16B14CD",
      "dimension_types": [1, 1, 1],
      "nperiodic_dimensions": 3,
      "lattice_vectors": [[8.59318, 0, 0], [0, 8.59318, 0], [0, 0, 4.41201]],
      "cartesian_site_positions": [[0, 0, 0], [0, 4.29659, 0], [4.29659, 0, 0], [0, 2.1438, 2.20601], [0, 6.44938, 2.20601], [4.29659, 2.13079, 2.20601], [4.29659, 6.46238, 2.20601], [2.1438, 0, 2.20601], [2.13079, 4.29659, 2.20601], [6.44938, 0, 2.206